# 1 - Preprocessing and module detection

## Ok, first thing is to generate the Count Matrix and QC
I'm going to work from raw data so the first thing I have to do is map Illumina reads for each sample to the zebrafinch genome and aggreate these mappings to the level of genes. 


I can't do this in a Jupyter notebook. I gathered the relevant set of HaNa's data and ran it through (the CountMatrix pipeline)[https://github.com/mattisabrat/CountMatrix/] (cloned 3/17/2020) using the following command:

    ./CountMatrix.sh -e /v-data2/matt_davenport/hana_reanalysis/hana_cm_formatted/ -t -n 8

the following config file:

    STAR_index_flags :
    STAR_quant_flags :
    FeatureCounts_aggregate_flags : minFragLength = 50
 
    Trimmomatic_flags :  CROP:95 HEADCROP:15 SLIDINGWINDOW:4:20 MINLEN:50
 
    Multiqc_flags : --interactive
    
and this genome and annotation: 
    
    ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/008/822/105/GCF_008822105.2_bTaeGut2.pat.W.v2 
    
with all entries missing a gene_id (tRNAs, mitogenes, etc.) purged from the .gtf using grep because that hangs up featurecounts.

### Set up env

In [ ]:
# Specify the lib path
lib_path<-paste(getwd(),'/hana_reanalysis_lib',sep='')
print(lib_path)

# Set the path
.libPaths(lib_path)
.libPaths()

### Only run the import if first time running. Takes forever

In [ ]:
# Import packages
#install.packages(c("tidyverse", "BiocManager", "matrixStats", "Hmisc",
#                   "foreach", "doParallel", "fastcluster", "dynamicTreeCut",
#                   "survival", "ggpubr", "ggdendro"),
#                lib=lib_path) 

#library(BiocManager)
#BiocManager::install(c("GO.db", "WGCNA", "preprocessCore", "impute", "DESeq2"),
#                    lib=lib_path)

### Read in data

In [ ]:
# Read in data
count_matrix <- readRDS('/v-data2/matt_davenport/hana_reanalyses/hana_cm_formatted/Count_Matrix.rds')

In [ ]:
# Lets take a look at the data
names(count_matrix)
print('counts')
dim(count_matrix$counts)
head(count_matrix$counts)
print('stat')
dim(count_matrix$stat)
head(count_matrix$stat)
print('annotation')
dim(count_matrix$annotation)
head(count_matrix$annotation)

### Convert to FPKM.

In [ ]:
#libs for wrangling
library(tidyverse)

#First lets reorder the row of annotation and counts to be alphabetical by gene
count_matrix$annotation <- count_matrix$annotation %>% arrange(GeneID)


#Put the gene_ids into a column of counts
count_matrix$counts$gene_id <- rownames(count_matrix$counts)
count_matrix$counts         <- count_matrix$counts %>% arrange(gene_id)

#Double check that they're identical
count_matrix$counts$gene_id[1:10]
count_matrix$annotation$GeneID[1:10]

#fraction identical
sum(count_matrix$counts$gene_id == count_matrix$annotation$GeneID) / length(count_matrix$annotation$GeneID)

#Move the lengths into counts
count_matrix$counts$lengths <- count_matrix$annotation$Length
head(count_matrix$counts)


In [ ]:
#convert
count_matrix$fpkm <- count_matrix$counts %>% 
                        transmute_at(vars(-lengths,-gene_id), 
                                     ~ . / ((sum(.)/10^6) * (lengths/10^3)))

#move over gene_ids
count_matrix$fpkm$gene_id <- count_matrix$counts$gene_id

head(count_matrix$fpkm)

In [ ]:
count_matrix$fpkm %>% filter(gene_id=='MEF2C') %>% t

#### Ok looks good

In [ ]:
#load the libs for analysis
library(WGCNA)
options(stringsAsFactors = FALSE)
disableWGCNAThreads()

### Format for WGCNA

In [ ]:
#create a list of things for WGCNA including a formated fpkm matrix. (genes as columns, samples as rows)
wgcna_tbl <- list()
wgcna_tbl$fpkm <- count_matrix$fpkm %>%
                        select(-one_of("gene_id")) %>%
                        t() %>% as.data.frame

colnames(wgcna_tbl$fpkm) <- count_matrix$counts$gene_id[]

head(wgcna_tbl$fpkm)

### Gene QC

In [ ]:
#now we need to kleen house hunty
#are there bad genes or samples?
wgcna_tbl$gsg <- goodSamplesGenes(wgcna_tbl$fpkm, verbose = 3);
wgcna_tbl$gsg$allOK

In [ ]:
#How many bad genes/samples are there
sum(wgcna_tbl$gsg$goodGenes==FALSE)
sum(wgcna_tbl$gsg$goodSamples==FALSE)


In [ ]:
#Ok, thats not bad at all, lets remove those genes.
wgcna_tbl$fpkm_gsg_filtered <- wgcna_tbl$fpkm %>% select(which(wgcna_tbl$gsg$goodGenes==TRUE))
wgcna_tbl$fpkm_gsg_rejects  <- wgcna_tbl$fpkm %>% select(which(wgcna_tbl$gsg$goodGenes==FALSE))

#make sure its the right size
print(length(count_matrix$counts$gene_id)-250)
print(length(colnames(wgcna_tbl$fpkm_gsg_filtered)))

#take a look at the kept data
head(wgcna_tbl$fpkm_gsg_filtered)

#list all the rejected genes
print(colnames(wgcna_tbl$fpkm_gsg_rejects))

### Sample clustering to find outliers

In [ ]:
sampleTree = hclust(dist(wgcna_tbl$fpkm_gsg_filtered), method = "average")
par(cex = 0.6);
par(mar = c(0,4,2,0))
plot(sampleTree, main = "Sample clustering to detect outliers", sub="", xlab="", cex.lab = 1.5, 
     cex.axis = 1.5, cex.main = 2, hang=-1)

### Two obviously offending samples, fv_ra_2 and mv_hvc_1, lets remove them

In [ ]:
wgcna_tbl$fpkm_gsg_filtered$sample_id <- rownames(wgcna_tbl$fpkm_gsg_filtered)
wgcna_tbl$fpkm_gsg_hclust_filtered <- wgcna_tbl$fpkm_gsg_filtered %>%
                                        filter(!sample_id %in% c("fv_ra_2", "mv_hvc_1"))

rownames(wgcna_tbl$fpkm_gsg_hclust_filtered) <- wgcna_tbl$fpkm_gsg_hclust_filtered$sample_id
wgcna_tbl$fpkm_gsg_hclust_filtered$sample_id <- NULL

#make sure we lost two samples
dim(wgcna_tbl$fpkm_gsg_hclust_filtered)

In [ ]:
#look at it again
sampleTree2 = hclust(dist(wgcna_tbl$fpkm_gsg_hclust_filtered), method = "average")
par(cex = 0.6);
par(mar = c(0,4,2,0))
plot(sampleTree2, main = "Sample clustering to detect outliers", sub="", xlab="", cex.lab = 1.5, 
     cex.axis = 1.5, cex.main = 2, hang=-1)

### Better, wish I hadn't had to ditch samples in the production pathway tho...

### Select tree trimming power for wgcna 
Constructing  a  weighted  gene  network  entails  the  choice  of  the  soft  thresholding  power β to  which  co-expressionsimilarity is raised to calculate adjacency. Here I use the function pickSoftThreshold to perform the analysis of network topology to help choose a proper soft-thresholding power. We want the lowest power which demonstrates scale free topolgy. <- paraphrased from docs

In [ ]:
# Choose a set of soft-thresholding powers
powers = c(1:20)

# Call the network topology analysis function
sft = pickSoftThreshold(wgcna_tbl$fpkm_gsg_hclust_filtered,
                        powerVector = powers,
                        verbose = 5,
                        blockSize=length(colnames(wgcna_tbl$fpkm_gsg_hclust_filtered)))

# Plot the results:
#sizeGrWindow(9, 5)
par(mfrow = c(1,2));
cex1 = 0.9;

# Scale-free topology fit index as a function of the soft-thresholding power
plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
     xlab="Soft Threshold (power)",ylab="Scale Free Topology Model Fit,signed R^2",type="n",
     main = paste("Scale independence"));
text(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
     labels=powers,cex=cex1,col="red");

# this line corresponds to using an R^2 cut-off of h
abline(h=0.90,col="red")

# Mean connectivity as a function of the soft-thresholding power
plot(sft$fitIndices[,1], sft$fitIndices[,5],
     xlab="Soft Threshold (power)",ylab="Mean Connectivity", type="n",
     main = paste("Mean connectivity"))
text(sft$fitIndices[,1], sft$fitIndices[,5], labels=powers, cex=cex1,col="red")

### I'm gonna use 6, its on the knee

### Detect modules

In [ ]:
#yippee ki yay
#Start with the most finescale I'll do, mainly just to get the TOM
wgcna_tbl$net_2_0.2 <- blockwiseModules(wgcna_tbl$fpkm_gsg_hclust_filtered, power = 6,
                                 maxBlockSize = length(colnames(wgcna_tbl$fpkm_gsg_hclust_filtered)),
                                 TOMType = "unsigned",
                                 minModuleSize = 100,
                                 deep_split = 2,       #these are the knobs
                                 mergeCutHeight = 0.2, #I'm going to fiddle with
                                 numericLabels = TRUE, 
                                 pamStage = FALSE, #and this
                                 saveTOMs = TRUE,
                                 saveTOMFileBase = "hana_wgcna_fpkm_gsg_hclust_filtered",
                                 corType = "pearson",
                                 verbose = 3,
                                 randomSeed = 42)

In [ ]:
#Make a dataframe of all the params I want to explore
param_list <- list()
param_list$deep_splits <- c(0,1,2)
param_list$mergeCutHeights <- c(0.2, 0.4, 0.6, 0.8, 0.9, 0.95)
param_list$PAM_step <-c(TRUE,FALSE)


param_tbl <- cross_df(param_list) %>% t %>% as.data.frame
colnames(param_tbl) <- lapply(param_tbl,function(x){return(paste(x[1],x[2],x[3],sep='_'))})
param_tbl$param <- row.names(param_tbl)
param_tbl <- param_tbl %>% as_tibble
param_tbl

In [ ]:
#Iterate using that TOM I saved above. 
nets <- lapply(param_tbl %>% select(-one_of('param')),
               function(x){
                   print(x)
                   net<-recutBlockwiseTrees(wgcna_tbl$fpkm_gsg_hclust_filtered,
                                       goodSamples=wgcna_tbl$net_2_0.2$goodSamples,
                                       goodGenes=wgcna_tbl$net_2_0.2$goodGenes,
                                       blocks=wgcna_tbl$net_2_0.2$blocks,
                                       TOMFiles=wgcna_tbl$net_2_0.2$TOMFiles,
                                       dendrograms=wgcna_tbl$net_2_0.2$dendrograms,
                                       TOMType = "unsigned",
                                       minModuleSize = 100,
                                       deep_split = x[1],       #these are the knobs
                                       mergeCutHeight = x[2], #I'm going to fiddle with
                                       numericLabels = TRUE, 
                                       pamStage = x[3], #and this
                                       corType = "pearson",
                                       verbose = 3,
                                       randomSeed = 42)
                   
                   return(net)
                                       
               })


In [ ]:
names(nets)
names(nets$`0_0.2_1`)

## This seems like a nice end point for this notebook. Lets gather the data we'll need moving forwards and save the data so we can start from here.

In [ ]:
#Add the CountMatrix to the nets lsit 
nets$input_data$fpkm        <- wgcna_tbl$fpkm_gsg_hclust_filtered
nets$input_data$unrecut_net <- wgcna_tbl$net_2_0.2

saveRDS(nets,'1_wgcna_recut_nets.rds')

# 2 - Model Selection and Cleaning

## Goal is to just to visualize each of our generated gene networks to select to one we feel best about for statistical testing. 


## Intro stuff

In [ ]:
# Specify the lib path
lib_path<-paste(getwd(),'/hana_reanalysis_lib',sep='')
print(lib_path)

# Set the path
.libPaths(lib_path)
.libPaths()

library(tidyverse)
library(WGCNA)

In [ ]:
#REad in the nets
nets <- readRDS('1_wgcna_recut_nets.rds')
names(nets)

In [ ]:
#Get input data out of the list
input_data      <- nets$input_data
nets$input_data <- NULL

names(nets$'0_0.2_1')



## Do some visualizing, we want the most modules without over fitting. If anything I'd like to be slightly overfit and I can then manually remove the overfit. 

In [ ]:
net <- nets$'2_0.6_1'

table(net$colors)

heatmap(net$MEs %>% data.matrix,scale="column")
heatmap(net$MEs %>% dist %>% data.matrix,scale="none",sym=1)
heatmap(net$MEs %>% t %>%  dist %>% data.matrix,scale="none",sym=1)

mergedColors = labels2colors(net$colors)

hist(net$colors,
    breaks=length(unique(net$colors)))
    
# Plot the dendrogram and the module colors underneath
plotDendroAndColors(input_data$unrecut_net$dendrograms[[1]], 
    mergedColors[input_data$unrecut_net$blockGenes[[1]]],
    'module_colors',
    dendroLabels = FALSE, hang = 0.03,
    addGuide = TRUE, guideHang = 0.05)

### OK I like 2_.6_1, but ME7 and ME13 are clearly fitting ingle samples

In [ ]:
net$MEs$ME13 <-NULL
net$MEs$ME7 <- NULL
net$colors[which(net$colors==13)] <-0
net$colors[which(net$colors==7)] <-0
table(net$colors)

### Vis again without the overfit modules

In [ ]:
heatmap(net$MEs %>% data.matrix,scale="column")
heatmap(net$MEs %>% dist %>% data.matrix,scale="none",sym=1)
heatmap(net$MEs %>% t %>%  dist %>% data.matrix,scale="none",sym=1)

mergedColors = labels2colors(net$colors)

hist(net$colors,
    breaks=length(unique(net$colors)))
    
# Plot the dendrogram and the module colors underneath
plotDendroAndColors(input_data$unrecut_net$dendrograms[[1]], 
    mergedColors[input_data$unrecut_net$blockGenes[[1]]],
    'module_colors',
    dendroLabels = FALSE, hang = 0.03,
    addGuide = TRUE, guideHang = 0.05)

In [ ]:
net$input_data <- input_data

## BEAUTIFUL
## Seems like a good place to save my work

In [ ]:
saveRDS(net, '2_wgcna_selected_purged_net.rds')

# 3 - Module Eigengene Assocation

In [ ]:
# Specify the lib path
lib_path<-paste(getwd(),'/hana_reanalysis_lib',sep='')
print(lib_path)

# Set the path
.libPaths(lib_path)
.libPaths()

### Import

In [ ]:
library(tidyverse)
library(WGCNA)

### Read in data. Read in experimental design. 

In [ ]:
net <-readRDS('wgcna_selected_purged_net.rds')
hana_design <- read.csv('hana_variables_by_sample.csv', header=TRUE, stringsAsFactors=FALSE)

#remove those samples I had to remove before
hana_design <- hana_design %>% filter(!sample_id %in% c("fv_ra_2", "mv_hvc_1"))

head(hana_design)
head(net$MEs)

In [ ]:
net$input_data$unrecut_net$TOMFiles
Tom<- load(net$input_data$unrecut_net$TOMFiles)

### Join and make it long form

In [ ]:
net$MEs$ME0 <- NULL
net$MEs$sample_id <- rownames(net$MEs)
me_tbl   <- left_join(net$MEs, hana_design, by='sample_id') %>% 
    pivot_longer(cols=-one_of(colnames(hana_design)),
                 names_to='me',values_to="me_expression")
head(me_tbl) 
unique(me_tbl$me)

### Make the binary variables numeric binaries and convert the ME identifiers to letters

In [ ]:
me_tbl <- me_tbl %>%
    mutate(is_song_cap = as.numeric(song_capable =='y')) %>%
    mutate(is_song_sys = as.numeric(song_system =='y')) %>%
    mutate(is_e2 = as.numeric(dose =='e')) %>%
    mutate(is_female = as.numeric(sex == 'f')) %>%
    mutate(me = recode_factor(me,
                                    'ME1'='A',
                                    'ME2'='B',
                                    'ME3'='C',
                                    'ME4'='D',
                                    'ME5'='E',
                                    'ME6'='F',

                                    'ME8'='G',
                                    'ME9'='H',
                                    'ME10'='I',
                                    'ME11'='J',
                                    'ME12'='K',

                                    'ME14'='L'
                             ))

head(me_tbl)

### Lets make all the subsets I think I might need

In [ ]:
#Prep a list of data subsets
tbls<-list()

tbls$all <- me_tbl
tbls$all_song_veh <- me_tbl %>% filter(is_e2==0) %>% filter(is_song_sys==1)
tbls$all_surr_veh <- me_tbl %>% filter(is_e2==0) %>% filter(is_song_sys==0)
tbls$m_tbl <- me_tbl %>% filter(is_female==0)
tbls$f_tbl <- me_tbl %>% filter(is_female==1)
tbls$f_noE_tbl <- me_tbl %>% filter(is_female==1) %>% filter(is_e2==0)
tbls$f_E_tbl <- me_tbl %>% filter(is_female==1) %>% filter(is_e2==1)
tbls$f_song_tbl <- me_tbl %>% filter(is_female==1) %>% filter(is_song_sys==1)
tbls$f_surround <- me_tbl %>% filter(is_female==1) %>% filter(is_song_sys==0)
tbls$m_veh_song <- me_tbl %>% filter(is_female==0) %>% filter(is_song_sys==1)
tbls$m_veh_surr <- me_tbl %>% filter(is_female==0) %>% filter(is_song_sys==0)



### Big thing to iterate over

In [ ]:
#Prep a df node x me combos
crs_lst <-list()
crs_lst$nodes <- unique(me_tbl$node) 
crs_lst$mes <- unique(me_tbl$me) 
node_me_tbl <- cross_df(crs_lst) %>% t %>% as.data.frame
colnames(node_me_tbl) <- lapply(node_me_tbl,function(x){return(paste(x[1],x[2],sep='_'))})

#make a long form for later use
node_tbl_long <- node_me_tbl %>% t %>% as.data.frame
node_tbl_long$name <- rownames(node_tbl_long)

head(node_me_tbl)

### Iterate and calculate MEG - trait correlations

In [ ]:
#Lets loop     
#first loop, data subset
res <- lapply(tbls,
             function(tbl){
                 #second loop, circuit node x ME combo
                 res <- lapply(node_me_tbl,
                               function(crs){
                                   
                                   #filter the table based on node x me
                                   tmp_tbl <- tbl %>% filter(node==crs[1],me==crs[2] %>% as.character)
                                   res<-list()
                                   res$song_cap_cor<- cor(tmp_tbl$me_expression, tmp_tbl$is_song_cap)
                                   res$song_sys_cor<- cor(tmp_tbl$me_expression, tmp_tbl$is_song_sys)
                                   res$sex_cor     <- cor(tmp_tbl$me_expression, tmp_tbl$is_female)
                                   res$e2_cor      <- cor(tmp_tbl$me_expression, tmp_tbl$is_e2)
                                   
                                   res$n_samp<-dim(tmp_tbl)[1]
                                   
                                   return(res)
                               }) 
                 return(res)
             })

In [ ]:
head(res$all$an_G)

## Time to wrangle
### I want each subset to become a long form tibble

In [ ]:
res2<- lapply(res,
            function(tbl){
                tmp <- tbl %>% 
                    as_tibble %>% mutate(measure=c('cap_cor','sys_cor','sex_cor','e2_cor','n'))
                res<-list()
                res$n_samps <- tmp %>% 
                    filter(measure=='n') %>% mutate(measure=NULL) %>%
                    pivot_longer(everything(),values_to='n') %>% mutate(n=as.numeric(n))
                
                res$cap_cor <- tmp %>% 
                    filter(measure=='cap_cor') %>% mutate(measure=NULL) %>%
                    pivot_longer(everything(),values_to='cap_cor') %>% 
                    mutate(cap_cor=as.numeric(cap_cor))

                res$sys_cor <- tmp %>% 
                    filter(measure=='sys_cor') %>% mutate(measure=NULL) %>%
                    pivot_longer(everything(),values_to='sys_cor')%>% 
                    mutate(sys_cor=as.numeric(sys_cor))
                
                res$sex_cor <- tmp %>% 
                    filter(measure=='sex_cor') %>% mutate(measure=NULL) %>%
                    pivot_longer(everything(),values_to='sex_cor')%>% 
                    mutate(sex_cor=as.numeric(sex_cor))
            
                res$e2_cor <- tmp %>% 
                    filter(measure=='e2_cor') %>% mutate(measure=NULL) %>%
                    pivot_longer(everything(),values_to='e2_cor') %>% 
                    mutate(e2_cor=as.numeric(e2_cor))
                
                res<-reduce(res,left_join, by='name') %>% 
                        mutate(node=substr(name,1,nchar(name)-2)) %>%
                        mutate(me=substr(name,nchar(name),nchar(name))) %>%
                        mutate(name=NULL)
                return(res)

                
            })

head(res2$all)

## Lovely, now lets calculate pvals within each node for each variable

In [ ]:
res3 <- lapply(res2,
              function(tbl){
                    holder <-lapply(unique(tbl$node),
                               function(nd){
                                tmp <- tbl %>% filter(node==nd) 
                                tmp$cap_p <- corPvalueStudent(tmp$cap_cor, n=tmp$n[1])
                                tmp$sys_p <- corPvalueStudent(tmp$sys_cor, n=tmp$n[1])
                                tmp$sex_p <- corPvalueStudent(tmp$sex_cor, n=tmp$n[1])
                                tmp$e2_p <- corPvalueStudent(tmp$e2_cor, n=tmp$n[1])
                                
                                return(tmp)
                                       })
                  
                    res<-accumulate(holder, union)[[4]]
                  
                    return(res)
                            })

res4<- res3 %>% lapply(mutate,is_cap=(cap_p<=0.05)) %>% lapply(mutate,is_sys=(sys_p<=0.05)) %>%
                lapply(mutate,is_sex=(sex_p<=0.05)) %>% lapply(mutate,is_e2=(e2_p<=0.05))
head(res4$all)


## Now lets geta separate table for each comparison

In [ ]:
res_tbls <- list()
res_tbls$cap <- res4 %>% lapply(select,c('n','cap_cor','cap_p','is_cap','me','node'))
res_tbls$sys <- res4 %>% lapply(select,c('n','sys_cor','sys_p','is_sys','me','node'))
res_tbls$sex <- res4 %>% lapply(select,c('n','sex_cor','sex_p','is_sex','me','node'))
res_tbls$e2  <- res4 %>% lapply(select,c('n','e2_cor','e2_p','is_e2','me','node'))

names(res_tbls)
names(res_tbls$cap)
head(res_tbls$cap$all)

In [ ]:
#read in and scrub the names`
res_tbls2 <- lapply(res_tbls, function(lst){
                    tbls <- lapply(lst,function(tbl){
                           names(tbl) <- c('n','cors','p','is_sig','me','node')
                           return(tbl)
                       })
                    return(tbls)
                   })
head(res_tbls2$cap$all)

In [ ]:
library(ggpubr)

In [ ]:
head(res_tbls2)

In [ ]:
#now to make plots
#function to grab the legend
g_legend<-function(a.gplot){
  tmp <- ggplot_gtable(ggplot_build(a.gplot))
  leg <- which(sapply(tmp$grobs, function(x) x$name) == "guide-box")
  legend <- tmp$grobs[[leg]]
  return(legend)} 
                      
nms <- names(res_tbls2)
names(nms) <- nms
plt_lst <- lapply(nms,
                  function(nm){

                      plts <- map(res_tbls2[[nm]],
                                    function(tbl){
                                        tbl <- tbl %>% mutate(me=factor(as.factor(me), 
                                                                       rev(c('A','B','C','D','E',
                                                                             'F','G', 'H', 'I', 'J', 'K', 'L'))))
                                        plt<- tbl %>% ggplot(aes(x=node,
                                                                 y=me,
                                                                 fill=node)) +
                                            geom_point(aes(size=cors^2,
                                                           alpha=-log10(p)),
                                                       pch=21) +
                                            geom_point(aes(size=cors^2,
                                                           color=is_sig, 
                                                           fill=NULL),
                                                       pch=21, stroke=3) +
                                            scale_alpha(trans='exp')+
                                            scale_fill_brewer(type='qual', palette='Dark2') +
                                            scale_size(name='r^2 ',range = c(1, 12)) + 
                                            scale_color_manual(name='p <= 0.05',values=c("grey","black")) +
                                            scale_x_discrete(labels = c("AN", "DN", "LA", "STR"))+
                                            theme_grey(base_size=25) +
                                            ylab('') + xlab('') + 
                                            guides(fill=FALSE, alpha=FALSE)+
                                            theme(legend.position="right",
                                                  legend.box = "vertical")
                                        return(plt)
                                    })
                      
                      lgnd <- g_legend(plts$all)
                      plts <- lapply(plts,function(plt){return(plt + theme(legend.position='none'))})
                      
                      plts$lgnd <- lgnd
                      #plts$dendro <- dendro
                      return(plts)
})

In [ ]:
options(repr.plot.width=15, repr.plot.height=8)


a<-plt_lst$cap$m_tbl +ggtitle('Male VL:\nSong Sys vs. Surround') + 
scale_x_discrete(labels=c('LMAN + AN','HVC + DN','RA + LAI','AX + Str')) + 
theme(axis.text.x=element_text(angle = 270, hjust = 0),plot.title =element_text(size=23) )


b<-plt_lst$sys$f_noE_tbl +ggtitle('Female Veh Song Sys:\nSong Sys vs. Surround') + 
scale_x_discrete(labels=c('LMAN + AN','HVC + DN','RA + LAI','AX + Str'))+
theme(axis.text.x=element_text(angle = 270, hjust=0),plot.title =element_text(size=23) )

top<-ggarrange(a,b,NULL,plt_lst$sex$lgnd,nrow=1,widths=c(1,1,.5,.5))

In [ ]:


a<-plt_lst$cap$f_tbl +ggtitle('Female VL: \nE2 Song Sys. vs Other') + 
scale_x_discrete(labels=c('LMAN + AN','HVC + DN','RA + LAI','AX + Str')) + 
theme(axis.text.x=element_text(angle = 270, hjust = 0),plot.title =element_text(size=23) )

b<-plt_lst$cap$all_song_veh +ggtitle('Veh VL + Sex:\nM vs. F Song Sys') + 
scale_x_discrete(labels=c('LMAN + AN','HVC + DN','RA + LAI','AX + Str'))+
theme(axis.text.x=element_text(angle = 270, hjust=0),plot.title =element_text(size=23) )

c<-plt_lst$sex$all_surr_veh +ggtitle('Veh Sex: \nM vs. F Surrounds') + 
scale_x_discrete(labels=c('LMAN + AN','HVC + DN','RA + LAI','AX + Str'))+
theme(axis.text.x=element_text(angle = 270, hjust=0),plot.title =element_text(size=23) )
options(repr.plot.width=13, repr.plot.height=16)

bottom<- ggarrange(a,b,c,nrow=1,widths=c(1,1,1))

ggarrange(top,bottom,nrow=2,heights = c(1,1))

## Now I need heatmaps of MEG expressions and rasters relating samples in those heatmaps to variables

In [ ]:
head(me_tbl)
unique(me_tbl$me)

In [ ]:
#Put the MEGs in order by size of M
me_tbl2 <- me_tbl %>% mutate(me_fct = factor(as.factor(me),
                                             c('A','B','C','D','E','F','G', 'H', 'I', 'J', 'K', 'L'))) %>% 
    mutate(me_int = substr(me,3,4) %>% strtoi )

head(me_tbl2)

In [ ]:
#Create a list of tbls subsetted by node
node_me_tbls <- me_tbl2 %>% group_by(node) %>% group_split
names(node_me_tbls) <- node_me_tbls %>% lapply(function(tbl){return(tbl$node[1])})
names(node_me_tbls) 

In [ ]:
#Create a list of list of sample names, arranged
node_me_tbls2 <- node_me_tbls %>% lapply(function(tbl){
    
    #arrange and get the order
    tbl <- tbl %>% arrange(desc(is_song_cap), desc(is_song_sys), desc(is_e2), desc(is_female), animal) 
    fct_order <- tbl$sample_id %>% unique
    
    return(tbl %>% 
               mutate(sample_id=factor(sample_id, fct_order)))
    
    
    })
head(node_me_tbls2$dn)

In [ ]:
#handle the node specificstuff for coloring.
#I need the vector of mod sizes 
'%!in%' <- function(x,y)!('%in%'(x,y))
mod_size_tbl <- table(net$colors) %>% as.data.frame %>% as_tibble %>% filter(Var1 %!in% c('0','7','13'))

mod_size_tbl <- mod_size_tbl %>%
    mutate(Var1 = factor(Var1,(unique(mod_size_tbl$Var1)))) %>%
    mutate(me = recode_factor(mod_size_tbl$Var1,
                                   '1'='A',
                                    '2'='B',
                                    '3'='C',
                                    '4'='D',
                                    '5'='E',
                                    '6'='F',

                                    '8'='G',
                                    '9'='H',
                                    '10'='I',
                                    '11'='J',
                                    '12'='K',

                                    '14'='L')) %>%
    mutate(me = factor(me,unique(me)))

head(mod_size_tbl)

### Yeah, I should've made a function for this, but I was having a bad day and couldn't be bothered so I copied and pasted

In [ ]:
plts_lst <- list()


    
me_size <- list()
me_size$me_size <- mod_size_tbl %>% ggplot(aes(x=Freq, y=me))+
                    geom_bar(stat='identity',aes(fill=me))+
                    scale_fill_brewer(palette = 'Set3') +
                    scale_fill_manual(values = c('black')) +
                    geom_label(aes(x=2500,label=Freq,fill=me),size=8)+
                    scale_fill_brewer(palette = 'Set3') +
                    scale_y_discrete(limits = rev(levels(mod_size_tbl$me))) +
                    theme_grey(base_size=25) + 
                    ggtitle('# Genes\nin Module') +
                    scale_x_continuous(limits=c(0,5000)) +
                    ylab('') + xlab('') +
                    theme(axis.text.x=element_blank(),
                          axis.text.y=element_text(size=27),
                          axis.title.x=element_blank(),
                          axis.line=element_blank(),
                          axis.ticks=element_blank(),
                         legend.position='none')
 

plts_lst$an <- ggplot(node_me_tbls2$an, aes(y=me,x=sample_id, fill=me_expression)) + geom_tile() +
    scale_fill_gradient2(low = "black",
    mid = "#1b9e77",
    high = "white") + 
    theme_grey(base_size=25) +
    ggtitle('\nLMAN + AN') +
    scale_y_discrete(limits = rev(levels(node_me_tbls2$an$me))) +
    xlab('') + ylab('') + 
    theme(legend.text=element_blank(),
         legend.title=element_blank(),
         axis.title=element_blank(),
         axis.text = element_blank(),
         axis.ticks = element_blank())

plts_lst$dn <- ggplot(node_me_tbls2$dn, aes(y=me,x=sample_id, fill=me_expression)) + geom_tile() +
    scale_fill_gradient2(low = "black",
    mid = "#d95f02",
    high = "white") + 
    theme_grey(base_size=25) +
    scale_y_discrete(limits = rev(levels(node_me_tbls2$dn$me))) +
    ggtitle('\nHVC + DN') +
    xlab('') + ylab('') + 
    theme(legend.text=element_blank(),
         legend.title=element_blank(),
         axis.title=element_blank(),
         axis.text = element_blank(),
         axis.ticks = element_blank())

plts_lst$la <- ggplot(node_me_tbls2$la, aes(y=me,x=sample_id, fill=me_expression)) + geom_tile() +
    scale_fill_gradient2(low = "black",
    mid = "#7570b3",
    high = "white") +
    theme_grey(base_size=25) +
    scale_y_discrete(limits = rev(levels(node_me_tbls2$la$me))) +
    ggtitle('\nRA + LAI') +
    xlab('') + ylab('') + 
    theme(legend.text=element_blank(),
         legend.title=element_blank(),
         axis.title=element_blank(),
         axis.text = element_blank(),
         axis.ticks = element_blank())

plts_lst$str <- ggplot(node_me_tbls2$str, aes(y=me,x=sample_id, fill=me_expression)) +
    geom_tile() +
    scale_fill_gradient2(
        low = "black",
        mid = "#e7298a",
        high = "white") +
    theme_grey(base_size=25) +
    scale_y_discrete(limits = rev(levels(node_me_tbls2$str$me))) +
    ggtitle('\nAX + Str') +
    xlab('') + ylab('') + 
    theme(legend.text = element_blank(),
         legend.title=element_blank(),
         axis.title=element_blank(),
         axis.text = element_blank(),
         axis.ticks = element_blank())

#Get the legends
plts_lst_lgnds <- plts_lst %>% lapply(g_legend)

#Strip the legends 
plts_lst <- plts_lst %>% lapply(function(plt){return( plt+theme(legend.position='none'))}) 

options(repr.plot.width=20, repr.plot.height=10)

fig1_2_1 <- ggarrange(plotlist=c(me_size, plts_lst,plts_lst_lgnds),
                    widths=c(1,1,1,1,1,.1,.1,.1,.1), nrow=1) %>% 
                        annotate_figure(left = text_grob("Module EigenGene",rot = 90, 
                                        face = "bold", size = 30)
                        )

fig1_2_1

In [ ]:
options(repr.plot.width=3, repr.plot.height=3)

show(plts_lst_lgnds)

## Ok Now I need rasters. First I need to pivot each independ variable wide 
## So matirx with sample as col, var as row for each region.

In [ ]:
var_tbls<-lapply(node_me_tbls2,
                 function(tbl){
                     var_lst<- lapply(c('is_song_cap', 'is_song_sys', 'is_female', 'is_e2'),
                            function(nm){
                                tbl2 <- tbl %>% select(one_of(c('sample_id',nm))) %>% 
                                            distinct %>% as.data.frame
                                rownames(tbl2) <- tbl2$sample_id
                                tbl2$sample_id <- NULL
                                tbl2 <- tbl2 %>% t %>% as_tibble %>% mutate(variable=nm)
                                
                                return(tbl2)
                            })
                     var_tbl <- bind_rows(var_lst) %>% pivot_longer(cols= -one_of('variable'),
                                                                    names_to='sample_id',
                                                                    values_to='is_true') %>%
                                                    mutate(sample_id = factor(as.factor(sample_id),
                                                                             levels(tbl$sample_id))) %>%
                                                    mutate(variable = factor(as.factor(variable),
                                                                            c('is_female',
                                                                             'is_e2',
                                                                             'is_song_sys',
                                                                             'is_song_cap')))

                     return(var_tbl)
                 })

head(var_tbls$an)

In [ ]:
raster_lst <- lapply(var_tbls,
                    function(tbl){
                        plt<- tbl %>%
                            ggplot(aes(x=sample_id,y=variable,fill=as.logical(is_true))) +
                            geom_tile() +
                            theme_grey(base_size=25)+
                            scale_fill_manual(values = c("grey", "black")) +
                            theme(legend.title=element_blank(),
                              axis.title =element_blank(),
                              axis.text = element_blank(),
                              axis.ticks = element_blank())
                        return(plt)
                    })

raster_lst$lgnd <- g_legend(raster_lst$an)

var_lbl <- tibble(var=factor(as.factor(c('Vocal Learning:',
                'Song System:',
                'Estradiol Treated:',
                'Female:')), 
                c('Female:',
                'Estradiol Treated:',
                'Song System:',
                'Vocal Learning:')))

raster_lbl <- ggplot(var_lbl,aes(x=1,y=var)) +
        geom_text(aes(label=var), stat='identity',size=6.5,hjust=1) +
        theme_classic(base_size=25) +
         xlim(0, 1) +
        theme(legend.title=element_blank(),
            axis.title = element_blank(),
            axis.text = element_blank(),
            axis.ticks = element_blank(),
            axis.line = element_line(color='white'))


fig1_2_2 <- ggarrange(raster_lbl,
         raster_lst$an +theme(legend.position='none'),
         raster_lst$dn +theme(legend.position='none'),
         raster_lst$la +theme(legend.position='none'),
         raster_lst$str +theme(legend.position='none'),
         raster_lst$lgnd, 
         nrow=1, widths=c(1,1,1,1,1,.4)) %>% 
                        annotate_figure(left = text_grob("Factors",rot = 90, 
                                        face = "bold", size = 30))

options(repr.plot.width=20, repr.plot.height=10)
fig1_2 <- ggarrange(fig1_2_1,fig1_2_2,nrow=2,heights=c(1,.3))
fig1_2

### I'd like a nice dendrogram too

In [ ]:
library(RColorBrewer)

In [ ]:
unique(net$colors)
net$colors[which(net$colors>13)] <- net$colors[which(net$colors>13)] -1
net$colors[which(net$colors>7)] <- net$colors[which(net$colors>7)] -1
net$colors[which(net$colors==0)] <- 13

unique(net$colors)

In [ ]:
pal <- c(brewer.pal(12, 'Set3'),'#000000') 


mergedColors = labels2colors(net$colors, zeroIsGrey = TRUE,
                            colorSeq = pal)

options(repr.plot.width=10, repr.plot.height=7)

plotDendroAndColors(net$input_data$unrecut_net$dendrograms[[1]], 
    mergedColors[net$input_data$unrecut_net$blockGenes[[1]]],
    '', ylab='' , main='',
    axes=FALSE,
    dendroLabels = FALSE, hang = 0.01,
    addGuide = TRUE, guideHang = .01,
    setLayout = TRUE,
    autoColorHeight = FALSE, colorHeight=.25)


## Boom. I need to hack a legend for it, use the size plot

In [ ]:
options(repr.plot.width=2, repr.plot.height=5)

gene_tree_lgnd <- mod_size_tbl %>% ggplot(aes(x=Freq, y=me))+
                    geom_bar(stat='identity',aes(fill=me))+
                    scale_fill_brewer(name='Module',palette = 'Set3') +
                    theme_classic(base_size=25) + 
                    theme(legend.position='right') 

gene_tree_lgnd <- gene_tree_lgnd %>% g_legend %>% ggarrange
gene_tree_lgnd

### WOOO!!! I think that's a great place to wrap that up for now. Might come back and make some more heatmaps at some point, but boom. I think that all looks awesome. Lets add res to net (which has been slightly modified) and save the thing out.

In [ ]:
net$node_me_tbls2 <- NULL
net$MEG_res_tbls <- res_tbls2
net$node_me_tbls <- node_me_tbls2
names(net)

In [ ]:
saveRDS(net, '3_wgcna_net_associatedToTraits.rds')

## MEG a specialization to RA would probably com out if I paired the data or otherwise accounted for that outlier animal. Lets do a quick and dirty non-parametric test

In [ ]:
x <- net$node_me_tbls$la %>% filter(me=='A') %>% filter(is_female == 0) %>% filter(is_song_cap==1) %>% select(me_expression) 
y <- net$node_me_tbls$la %>% filter(me=='A') %>% filter(is_female == 0) %>% filter(is_song_cap==0) %>% select(me_expression)
wilcox.test(x=x%>%as.matrix,y=y%>%as.matrix)

# 4 - Defining Continuous Module Membership

In [ ]:
# Specify the lib path
lib_path<-paste(getwd(),'/hana_reanalysis_lib',sep='')
print(lib_path)

# Set the path
.libPaths(lib_path)
.libPaths()

In [ ]:
library(tidyverse)
library(WGCNA)

In [ ]:
net <- readRDS('3_wgcna_net_associatedToTraits.rds')
names(net)

## First, lets wrangle that colors thing into a nice tibble.

In [ ]:
gene_tbl <- net$colors %>% as.data.frame
colnames(gene_tbl) <- 'me_assigned'
gene_tbl$gene_id<- row.names(gene_tbl)
gene_tbl <- gene_tbl %>% as_tibble %>% mutate(me_assigned = recode_factor(as.factor(me_assigned),
                                                        '1'='A',
                                                        '2'='B',
                                                        '3'='C',
                                                        '4'='D',
                                                        '5'='E',
                                                        '6'='F',

                                                        '7'='G',
                                                        '8'='H',
                                                        '9'='I',
                                                        '10'='J',
                                                        '11'='K',

                                                        '12'='L',
                                                        '13'='X'))
head(gene_tbl)
levels(gene_tbl$me_assigned)

## Now get the expression of genes (row samples cols genes and MEGs)

In [ ]:
gene_mod_cor_tbl <- net$input_data$fpkm %>% as_tibble(rownames='sample_id') 
head(gene_mod_cor_tbl)

In [ ]:
mes <- net$MEs
names(mes)

In [ ]:
names(mes) = c('B', 'G', 'D', 'J', 'A', 'L', 'E', 'K', 'F', 'H', 'C', 'I', 'sample_id')
Mes <- mes %>% as_tibble(rownames='sample_id') 
gene_mod_cor_tbl <- left_join(mes,gene_mod_cor_tbl)
head(gene_mod_cor_tbl)

## Now I need to correlate each of the genes to each of the MEGs across samples
## I want a matrix with cols = genes, rows = MEGs, value is results of WGCNA cor 

In [ ]:
megs <-c('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L')
res <- gene_mod_cor_tbl %>% select(one_of(megs)) %>%
    lapply(function(meg){
        res <- gene_mod_cor_tbl %>% select(-one_of(c(megs,'sample_id'))) %>%
            lapply(function(gene){
                return(cor(gene,meg))
            })
        return(res)
    })

In [ ]:
res2 <- res %>% as_tibble 


In [ ]:
res3 <- res2 %>% mutate(gene_id = names(res2$A)) %>% mutate_at(megs,as.numeric) %>% left_join(gene_tbl)
head(res3)

In [ ]:
saveRDS(res3,'4_mod_membership_all_genes.rds')

# 5 - Module GO and Convergent Signature Enrichment 

In [ ]:
# Specify the lib path
lib_path<-paste(getwd(),'/hana_reanalysis_lib',sep='')
print(lib_path)

# Set the path
.libPaths(lib_path)
.libPaths()

In [ ]:
library(tidyverse)
library(ggpubr)

In [ ]:
mod_mem <- readRDS('mod_membership_all_genes.rds')
head(mod_mem)

In [ ]:
#BiocManager::install("gage")
#BiocManager::install("gageData")

library(gage)
library(gageData)

### Quick look

In [ ]:
assigned_genes <- mod_mem %>% filter(me_assigned != 'X') %>% dplyr::select('gene_id','me_assigned')
head(assigned_genes)
dim(assigned_genes)

## To do ontology I need to map z finch genes to human orthologs. FUN

In [ ]:
#BiocManager::install("org.Hs.eg.db")
#BiocManager::install("AnnotationDbi")
#BiocManager::install("biomaRt")
#BiocManager::install("GO.db")

library(GO.db)
library(org.Hs.eg.db)
library(AnnotationDbi)
library(biomaRt)

## Stolen from Greg

In [ ]:
getOrthos <- function(input_org,output_org,one2one) {
  tmp_input <- paste(input_org,"_gene_ensembl",sep = "")
  tmp_mart <- useMart("ensembl", dataset = tmp_input,host = "www.ensembl.org",ensemblRedirect = FALSE)
  tmp_attr <- listAttributes(tmp_mart)
  tmp_genes <- getBM(attributes = c("ensembl_gene_id", "gene_biotype"), mart = tmp_mart)
  tmp_genes <- tmp_genes[grep("protein_coding",tmp_genes$gene_biotype),]
  attr_names <- tmp_attr[grep(output_org,tmp_attr$name),"name"]
  tmp_orthos <- getBM(attributes = c("ensembl_gene_id","external_gene_name",attr_names), filters = "ensembl_gene_id",values = tmp_genes$ensembl_gene_id, mart = tmp_mart)  
  
  if(one2one == TRUE){
  tmp_orthos <- tmp_orthos[grep("ortholog_one2one",tmp_orthos[,paste(output_org,"_homolog_orthology_type",sep = "")]),]
  }

  if(one2one == FALSE){
    tmp_orthos <- tmp_orthos
  }
  
  tmp_orthos

}

### Make the conversion table

In [ ]:
convert_table <- getOrthos("tguttata","hsapiens",TRUE) %>% as_tibble %>% 
    dplyr::select(external_gene_name, hsapiens_homolog_associated_gene_name, hsapiens_homolog_ensembl_gene ) %>%
    mutate(gene_id = external_gene_name) %>% mutate(external_gene_name = NULL) %>%
    mutate(hsap_gene_id = hsapiens_homolog_associated_gene_name) %>% mutate(hsapiens_homolog_associated_gene_name = NULL) %>%
    mutate(hsap_ensembl_id = hsapiens_homolog_ensembl_gene) %>% mutate(hsapiens_homolog_ensembl_gene = NULL) 

head(convert_table)

In [ ]:
assigned_genes2 <- assigned_genes %>% left_join(convert_table, by='gene_id') %>% filter(hsap_ensembl_id != 'NA')
head(assigned_genes2)
dim(assigned_genes2)

## Ok now to get to the super informative GO stuff

In [ ]:
data(go.sets.hs)
data(go.subs.hs)

In [ ]:
mods <- unique(assigned_genes2$me_assigned %>% as.character)
names(mods) <- mods

gage_res_discrete <- mods %>% lapply(function(mod){

    gage_tbl <- assigned_genes2 %>% mutate(inModule = as.numeric(me_assigned==mod))  %>% 
                    dplyr::select(hsap_ensembl_id,inModule)

    gage_exprs <- gage_tbl$inModule %>% as.matrix
    rownames(gage_exprs) <- gage_tbl$hsap_ensembl_id
    return (gage(gage_exprs, gsets = go_terms, ref = NULL, samp = NULL)$greater %>%
                as.data.frame %>% as_tibble(rownames='GO_id')%>% filter(p.val<1))
})

## Lil wrangle

In [ ]:
gage_res_discrete2 <- mods %>% lapply(function(mod){
    gage_res_discrete[[mod]] %>% mutate(module=mod)
})

In [ ]:
gage_p_arranged <- gage_res_discrete2 %>% lapply(arrange,p.val)
gage_p_arranged2 <- gage_p_arranged %>% lapply(function(tbl){
    tbl %>% mutate(GO_id=factor(as.factor(GO_id),levels=unique(tbl$GO_id)%>%rev))
})

In [ ]:
sig_G_go<-gage_p_arranged2$G %>% filter(p.val<=0.05)
sig_E_go<-gage_p_arranged2$E %>% filter(p.val<=0.05)

### How's it look?

In [ ]:
sig_G_go
sig_E_go

## Need to get go terms, not just IDs. What a pain

In [ ]:
library(GO.db)

# extract a named vector of all terms
goterms <- Term(GOTERM)
GOids <- names(goterms)

GO_terms_tbl <- tibble(GOterms=goterms,GOids=GOids)

In [ ]:
sig_G_go2 <- sig_G_go %>% mutate(GOids = GO_id) %>% mutate(GO_id=NULL) %>% left_join(GO_terms_tbl,by='GOids')

sig_E_go2 <- sig_E_go %>% mutate(GOids = GO_id) %>% mutate(GO_id=NULL) %>% left_join(GO_terms_tbl,by='GOids')

In [ ]:
sig_G_go3 <- sig_G_go2 %>% mutate(p=p.val)%>% mutate(p=signif(p,2)) %>% mutate(GOid=GOids) %>%
    mutate(GOterm=GOterms)%>%dplyr::select(GOterm,GOid,p)
sig_E_go3 <- sig_E_go2 %>% mutate(p=p.val)%>% mutate(p=signif(p,2)) %>% mutate(GOid=GOids) %>%
    mutate(GOterm=GOterms)%>%dplyr::select(GOterm,GOid,p)

sig_G_go3
sig_E_go3

### Make a pretty table using below code, fiddled with here: https://www.w3schools.com/html/html_tables.asp

In [ ]:
library(stargazer)

In [ ]:
star1<-stargazer(sig_G_go3,type='html',summary = FALSE,rownames=FALSE,title='Module G: Significantly Enriched GO')

## Now I gotta do something similar for the RA and HVC convergent gene signatures to LMC. Probably want some kind of bubble plot.

In [ ]:
ra_raw <- read.csv2(file='ra_lmc_sharedGenes.csv',header = TRUE,sep = ',')
hvc_raw <- read.csv2(file='hvc_lmc_sharedGenes.csv',header = TRUE,sep = ',')


In [ ]:
greg_enrich_list <- list()
greg_enrich_list$RA <- ra_raw$gene %>% as.character
greg_enrich_list$HVC <- hvc_raw$gene %>% as.character

greg_enrich_list$RA
greg_enrich_list$HVC

In [ ]:
gage_res_greg_convergent <- mods %>% lapply(function(mod){

    gage_tbl <- assigned_genes2 %>% mutate(inModule = as.numeric(me_assigned==mod))  %>% 
                    dplyr::select(hsap_ensembl_id,inModule)

    gage_exprs <- gage_tbl$inModule %>% as.matrix
    rownames(gage_exprs) <- gage_tbl$hsap_ensembl_id
    return (gage(gage_exprs, gsets = greg_enrich_list, ref = NULL, samp = NULL)$greater %>%
                as.data.frame %>% as_tibble(rownames='GO_id')#%>% filter(p.val<1)
           )
})

gage_res_greg_convergent2<- mods %>% lapply(function(mod){
    gage_res_greg_convergent[[mod]] %>% mutate(module=mod)
}) %>% purrr::reduce(rbind)



In [ ]:
head(gage_res_greg_convergent2 )

### Erich wants to see the actual enrichment value for this

In [ ]:
convergent_fold_enrichment <- mods %>% lapply(function(mod){
    n_hvc_convergent <- length(greg_enrich_list$HVC)
    n_ra_convergent  <- length(greg_enrich_list$RA)
    
    total_genes <- assigned_genes2$gene_id %>% length
    
    tmp_tbl <- assigned_genes2 %>% filter(me_assigned == mod)
    mod_total <- tmp_tbl$gene_id %>% length
    
    ra_tbl <- tmp_tbl %>% filter(hsap_ensembl_id %in% greg_enrich_list$RA)
    n_ra <- ra_tbl$gene_id %>% length
    hvc_tbl <- tmp_tbl %>% filter(hsap_ensembl_id %in% greg_enrich_list$HVC)
    n_hvc <- hvc_tbl$gene_id %>% length
    
    expected_fraction_ra  <- (n_ra_convergent / total_genes) * (mod_total / total_genes)
    expected_fraction_hvc <- (n_hvc_convergent / total_genes) * (mod_total / total_genes)

    observed_fraction_ra  <- n_ra / total_genes
    observed_fraction_hvc <- n_hvc / total_genes
    res <- tibble(GO_id = c('RA','HVC'),
                  module = c(mod,mod),
                  fold_expected = c(observed_fraction_ra/expected_fraction_ra,
                                    observed_fraction_hvc/expected_fraction_hvc),
                  mod_size=mod_total)
    return(res)
    
}) %>% purrr::reduce(rbind)

In [ ]:
gage_res_greg_convergent3 <- gage_res_greg_convergent2 %>% 
    left_join(convergent_fold_enrichment, by=c('module','GO_id')) %>% mutate(is_sig=q.val<=0.1)

In [ ]:
#function to grab the legend
g_legend<-function(a.gplot){
  tmp <- ggplot_gtable(ggplot_build(a.gplot))
  leg <- which(sapply(tmp$grobs, function(x) x$name) == "guide-box")
  legend <- tmp$grobs[[leg]]
  return(legend)} 

In [ ]:
options(repr.plot.width=10, repr.plot.height=9)

converge_plter <- function(reg){
    gage_res_greg_convergent3 %>% filter(GO_id==reg) %>%
    ggplot(aes(x=mod_size,
               y=fold_expected,
               fill=module,color=is_sig, 
               size = -log10(p.val)))+
        geom_hline(yintercept=1,color='black',size=2) + 
        geom_point(data=gage_res_greg_convergent3%>% filter(GO_id==reg) %>% filter(is_sig==FALSE),shape=21,stroke=2) + 
        geom_point(data=gage_res_greg_convergent3%>% filter(GO_id==reg) %>% filter(is_sig==TRUE),shape=21,stroke=2) + 
        scale_fill_brewer('Module',palette='Set3')+
        scale_color_manual('p <= 0.05', values=c('grey','black'))+
        scale_size_continuous('-log10(p)', range=c(1,10), breaks=c(0,1.25,2.5),limits=c(0,2.5))+
        theme_grey(base_size=25) +
        scale_y_continuous('Fold Enrichment',breaks=c(0,.5,1,1.5,2),limits=c(0,2))+
        scale_x_continuous('')+ 
        ggtitle(reg)+
        guides(fill = guide_legend(override.aes = list(size=10)))}

hvc_plt <- converge_plter('HVC') + ggtitle('HVC convergent specialization')
lgnd<- g_legend(hvc_plt)
ra_plt <- converge_plter('RA') +theme(legend.position='none') +scale_x_continuous('Module Size') + ggtitle("RA convergent specialization")
hvc_plt <- hvc_plt+theme(legend.position='none')+xlab('')


col1 <- ggarrange(hvc_plt,ra_plt,nrow=2,heights = c(1,1.05))
ggarrange(col1,lgnd,widths=c(1,.5))

### Who are these genes in B?

In [ ]:
assigned_genes2 %>% filter(me_assigned == 'B') %>% filter(hsap_ensembl_id %in% greg_enrich_list$HVC)

# 6 - Core gene analysis

In [ ]:
# Specify the lib path
lib_path<-paste(getwd(),'/hana_reanalysis_lib',sep='')
print(lib_path)

# Set the path
.libPaths(lib_path)
.libPaths()

In [ ]:
library(tidyverse)
library(WGCNA)
library(ggpubr)

In [ ]:
nets <- readRDS('1_wgcna_recut_nets.rds')
fpkm <- nets$input_data$fpkm %>% as_tibble(rownames='sample_id')

hana_design <- read.csv('hana_variables_by_sample.csv', header=TRUE, stringsAsFactors=FALSE)

#remove those samples I had to remove before
hana_design <- hana_design %>% filter(!sample_id %in% c("fv_ra_2", "mv_hvc_1")) %>% as_tibble

In [ ]:
names(nets$input_data)

In [ ]:
head(fpkm)
head(hana_design)

In [ ]:
fpkm <- fpkm %>% pivot_longer(-one_of('sample_id'),
                              names_to='gene_id',
                              values_to='fpkm')
head(fpkm)

In [ ]:
fpkm <- left_join(hana_design,fpkm,by='sample_id')
head(fpkm)

## Now the mod memberships

In [ ]:
mod_mem <- readRDS('4_mod_membership_all_genes.rds')
head(mod_mem)

In [ ]:
mod_mem <- mod_mem %>% pivot_longer(-one_of(c('gene_id','me_assigned')),
                                    names_to='module',
                                    values_to='membership')
head(mod_mem)

## First thing is to make a list of tibbles, subset from fpkm. One for each set within which I'm computing gene significance.

In [ ]:
gen_list <- list()

gen_list$m_an <- fpkm %>% filter(sex=='m') %>% filter(node=='an')
gen_list$f_an <- fpkm %>% filter(sex=='f') %>% filter(node=='an')

gen_list$m_dn <- fpkm %>% filter(sex=='m') %>% filter(node=='dn')
gen_list$f_dn <- fpkm %>% filter(sex=='f') %>% filter(node=='dn')

gen_list$m_la <- fpkm %>% filter(sex=='m') %>% filter(node=='la')
gen_list$f_la <- fpkm %>% filter(sex=='f') %>% filter(node=='la')

gen_list$m_str <- fpkm %>% filter(sex=='m') %>% filter(node=='str')
gen_list$f_str <- fpkm %>% filter(sex=='f') %>% filter(node=='str')

gen_list$f_an_v <- fpkm %>% filter(sex=='f') %>% filter(node=='an') %>% filter(dose=='v')
gen_list$f_dn_v <- fpkm %>% filter(sex=='f') %>% filter(node=='dn') %>% filter(dose=='v')  
gen_list$f_la_v <- fpkm %>% filter(sex=='f') %>% filter(node=='la') %>% filter(dose=='v')  
gen_list$f_str_v <- fpkm %>% filter(sex=='f') %>% filter(node=='str') %>% filter(dose=='v')  

gen_list$fm_lman_v <- fpkm %>% filter(region=='lman') %>% filter(dose=='v')
gen_list$fm_hvc_v <-  fpkm %>% filter(region=='hvc') %>% filter(dose=='v')
gen_list$fm_ra_v <-   fpkm %>% filter(region=='ra') %>% filter(dose=='v')
gen_list$fm_ax_v <-   fpkm %>% filter(region=='ax') %>% filter(dose=='v')

gen_list$f_lman <- fpkm %>% filter(region=='lman') %>% filter(sex=='f')
gen_list$f_hvc <-  fpkm %>% filter(region=='hvc') %>% filter(sex=='f')
gen_list$f_ra <- fpkm %>% filter(region=='ra') %>% filter(sex=='f')
gen_list$f_ax <- fpkm %>% filter(region=='ax') %>% filter(sex=='f')



names(gen_list)
head(gen_list$fm_ax_v)




In [ ]:
#I need a list of all the genes 
genes <- unique(gen_list$m_an$gene_id)
names(genes) <- genes

## Do a loop run the stats. Doing it in one shot with corAndP

In [ ]:
a <- gen_list %>% lapply(function(sample_set){
    res_tbl <- genes %>% lapply(function(gene){
        x <- sample_set %>% filter(gene_id == gene)
        corAndPvalue(x$fpkm, x$song_capable=='y') %>% lapply(as.numeric)
    }) 
})

### wrangle into a nice tibble, takes a while

In [ ]:
song_cap_res_tbls <- a %>% lapply(function(tbl){
    
    res <- tbl %>% t
    res <-res %>% lapply(as.data.frame)
    names(res) <- names(tbl)
    res <- names(res) %>% lapply(function(ge){res[[ge]] %>% mutate(gene_id=ge)})
    res <- res %>% purrr::reduce(rbind) %>% mutate(q=p.adjust(p, method='fdr'))
})

head(song_cap_res_tbls$f_an)

In [ ]:
song_cap_res_tbls <- song_cap_res_tbls %>% lapply(left_join,mod_mem,by='gene_id')


In [ ]:
head(song_cap_res_tbls$f_dn)

In [ ]:
options(repr.plot.width=12.5, repr.plot.height=10)
cor_cor_plot <- function(tbl,modu,rect_col){
tbl %>% filter(module==modu) %>% filter(me_assigned!='X') %>%
    
    ggplot(aes(x=membership,
               y=cor,
               fill=me_assigned,
               size=me_assigned==modu,
               color=me_assigned==modu)) + 
    geom_rect(aes(xmin = .5^.5, xmax = 1, ymin = .5^.5, ymax = 1),
               fill = rect_col, color='black', size = .75) +
    
    geom_point(shape=21)+
    xlim(-1,1) + ylim(-1,1)+
    scale_color_manual(values=c('grey','black'))+
    geom_smooth(method = "lm",color='black',fill='grey',size=2)+
    scale_fill_brewer('Module',palette = 'Set3')+
    scale_size_manual(values=c(2,3))+
    theme_grey(base_size=25) + 
    xlab(modu)+ylab('VL Capable')+guides(size=FALSE,color=FALSE)+
    guides(fill = guide_legend(override.aes = list(size=10)))
    
    }



In [ ]:
g_legend<-function(a.gplot){
  tmp <- ggplot_gtable(ggplot_build(a.gplot))
  leg <- which(sapply(tmp$grobs, function(x) x$name) == "guide-box")
  legend <- tmp$grobs[[leg]]
  return(legend)} 

m_dn_g<-cor_cor_plot(song_cap_res_tbls$m_dn,'G',"#1b9e77") + ggtitle('Male VL:\nHVC vs. DN') +theme(plot.title = element_text(size=30),legend.position='none')+ylab('Vocal Learning')

f_dn_g<-cor_cor_plot(song_cap_res_tbls$f_dn,'G',"#d95f02") + ggtitle('Female VL: \nE2-HVC vs. DN & Veh-HVC') +theme(plot.title = element_text(size=30),legend.position='none') +ylab('')

fm_hvc_v<-cor_cor_plot(song_cap_res_tbls$fm_hvc_v,'G',"#7570b3") + ggtitle('Untreated VL + Sex: \nMale-HVC vs. Female-HVC') +theme(plot.title = element_text(size=30),legend.position='none')+ylab('')

f_hvc<-cor_cor_plot(song_cap_res_tbls$f_hvc,'G',"#e7298a") + ggtitle('Female VL + E2: \nVeh-HVC vs. E2-HVC') +ylab('') +theme(plot.title = element_text(size=30))

lgnd <-  g_legend(f_hvc)

f_hvc <- f_hvc + theme(legend.position='none')

In [ ]:
options(repr.plot.width=28+7/4, repr.plot.height=7)
top<-ggarrange(m_dn_g,f_dn_g,fm_hvc_v,f_hvc,lgnd,nrow=1,widths=c(1,1,1,1,.25))

In [ ]:
library(ggrepel)

In [ ]:
options(repr.plot.width=12.5, repr.plot.height=10)

zoom_cor_cor_plot <- function(tbl,modu,bottom_left,top_right,rect_col){
    tbl %>% filter(module==modu) %>% filter(me_assigned!='X') %>% 
    
    ggplot(aes(x=membership,
               y=cor)) + 
    geom_label_repel(aes(label=gene_id),size=6,fill='white', color='black',max.overlaps = Inf)+
    xlim(bottom_left[1],top_right[1]) +
    ylim(bottom_left[2],top_right[2])+
    theme_grey(base_size=25) + 
    xlab(modu)+ylab('VL Capable')+
    guides(fill = guide_legend(override.aes = list(size=10)))+
    theme(panel.background = element_rect(fill =rect_col),panel.grid = element_blank())
}



In [ ]:
m_dn_zoom_p<-zoom_cor_cor_plot(song_cap_res_tbls$m_dn,'G',c(.5^.5,.5^.5),c(1,1),"#1b9e77") +theme(legend.position = 'none',axis.title=element_blank()) 
f_dn_zoom_p<-zoom_cor_cor_plot(song_cap_res_tbls$f_dn,'G',c(.5^.5,.5^.5),c(1,1),"#d95f02")+theme(legend.position = 'none',axis.title=element_blank()) 
fm_hvc_v_zoom_p<-zoom_cor_cor_plot(song_cap_res_tbls$fm_hvc_v,'G',c(.5^.5,.5^.5),c(1,1),"#7570b3")+theme(legend.position = 'none',axis.title=element_blank()) 
f_hvc_zoom_p<-zoom_cor_cor_plot(song_cap_res_tbls$f_hvc,'G',c(.5^.5,.5^.5),c(1,1),"#e7298a")+theme(legend.position = 'none',axis.title=element_blank()) 


bottom <- ggarrange(m_dn_zoom_p,f_dn_zoom_p,fm_hvc_v_zoom_p,f_hvc_zoom_p,nrow=2, ncol=2)

In [ ]:
options(repr.plot.width=30+7/4, repr.plot.height=8)
top

In [ ]:
options(repr.plot.width=15, repr.plot.height=15)
bottom


In [ ]:
zoom_list <- function(tbl,modu,bottom_left=c(-1,-1),top_right=c(1,1)){
    
    tbl %>% filter(module==modu) %>% filter(me_assigned!='X') %>% 
    filter(cor > bottom_left[2]) %>% filter(cor < top_right[2])%>% 
    filter(membership > bottom_left[1]) %>% filter(membership < bottom_left[1] + .5)
}

In [ ]:
m_dn_zoom_l<-zoom_list(song_cap_res_tbls$m_dn,'G',c(.5^.5,.5^.5),c(1,1))$gene_id
f_dn_zoom_l<-zoom_list(song_cap_res_tbls$f_dn,'G',c(.5^.5,.5^.5),c(1,1))$gene_id
fm_hvc_v_zoom_l<-zoom_list(song_cap_res_tbls$fm_hvc_v,'G',c(.5^.5,.5^.5),c(1,1))$gene_id 
f_hvc_zoom_l<-zoom_list(song_cap_res_tbls$f_hvc,'G',c(.5^.5,.5^.5),c(1,1))$gene_id

In [ ]:
library('VennDiagram')

In [ ]:
z_list <- rtracklayer::import('/v-data2/matt_davenport/hana_reanalyses/hana_cm_formatted/genome/GCF_008822105.2_bTaeGut2.pat.W.v2_genomic.purged2.gtf') %>%
            as_tibble %>% 
            filter(type=='gene') %>% dplyr::select(one_of(c('gene_id','start', 'seqnames'))) %>% arrange(start) %>%
            mutate(chr=seqnames) %>% mutate(seqnames=NULL) %>% filter(chr=='NC_045027.1')
z_list <- z_list$gene_id


In [ ]:
venn_lists <- list(m_dn_zoom_l,f_dn_zoom_l,fm_hvc_v_zoom_l,f_hvc_zoom_l)

names(venn_lists)<-c('Male VL\n','Female VL\t\n','VL + Sex\n','VL + E2\n')


In [ ]:
venn.diagram(x=venn_lists,filename = 'core_genes.png',output=TRUE,fill=RColorBrewer::brewer.pal(4, "Dark2"),
             height = 2100, width = 2100)

In [ ]:
core_genes <- Reduce(intersect,venn_lists)
all_assigned_genes <- song_cap_res_tbls$f_dn %>% filter(me_assigned!='X') %>% filter(module=='A')
all_assigned_genes<- all_assigned_genes$gene_id

assigned_z_list= intersect(z_list,all_assigned_genes)

core_z_venn_lists <- list(core_genes, assigned_z_list, all_assigned_genes)
names(core_z_venn_lists) <- c(' ', '', '  ')
venn.diagram(x=core_z_venn_lists, filename = 'core_Zchrom_intersect.png',
             output=TRUE,fill=c('#b000cf','#4a4a4a','light grey'),
             height = 800, width = 1200
)

## Statistcal testing for z chromosome enrichment in G core

### assigned genes

In [ ]:
phyper(q = 4,
       m = 820,
       n = length(all_assigned_genes)-820,
       k = 15, 
       lower.tail = FALSE, log.p = FALSE)

### module G genes

In [ ]:
mod_mem %>% filter(module=="G")%>% filter(me_assigned=='G') %>% filter(gene_id %in% z_list) %>% dim

In [ ]:
phyper(q=4,
       m=25,
       n=344-25,
       k=15,
       lower.tail = FALSE, log.p = FALSE)

### all genes

In [ ]:
phyper(q = 4,
       m = length(z_list),
       n = 21040-length(z_list),
       k = 15, 
       lower.tail = FALSE, log.p = FALSE)

## Lets just look at the expression of these genens

In [ ]:
options(repr.plot.width=30, repr.plot.height=15)

names(core_genes) <- core_genes
plts<-core_genes %>% lapply(function(gen){
    tmp<-fpkm %>% filter(node=='dn') %>% filter(gene_id==gen) %>% mutate(grouping = paste(sex,dose,region,sep='_')) 
    male_veh_mean <- tmp %>% filter(grouping=='m_v_hvc') 
    male_veh_mean <- mean(male_veh_mean$fpkm)
    tmp<-tmp %>% mutate(norm_expr = fpkm/male_veh_mean) %>% mutate(grouping=grouping%>%as.factor)
    
    tmp$grouping <- factor(tmp$grouping,c('m_v_hvc','m_e_hvc','m_v_pln','m_e_pln',
                                          'f_v_hvc','f_e_hvc','f_v_pln','f_e_pln'))
    
    ggplot(tmp,aes(x=grouping,y=norm_expr,fill=sex,size=song_capable)) + 
        geom_bar(stat = "summary",color='black')+scale_size_manual("VL Capable",values=c(0,1))+
        geom_point(shape=21,size=3,fill='grey') +theme_grey(base_size=25) + ggtitle(gen) + 
        xlab("") + ylab('') + scale_x_discrete(labels=c("M-Veh-HVC",'M-E2-HVC','M-Veh-DN','M-E2-DN',
                                                      "F-Veh-HVC",'F-E2-HVC','F-Veh-DN','F-E2-DN')) +
        theme(axis.text.x=element_text(angle = -90, hjust = 0))
    
})

lgnd <- g_legend(plts$GHR)

plts<-plts %>% lapply(function(plt){plt+theme(legend.position='none')})
row1<-ggarrange(plts$GHR,plts$LRRC2,plts$RGS7BP,plts$THBS4,plts$CENPE,NULL,nrow=1,widths=c(1,1,1,1,1,1))
row2<-ggarrange(plts$EDA2R,plts$FAM102B,plts$FBXL13,plts$LOC115494282,plts$LOC115494463,lgnd,nrow=1,widths=c(1,1,1,1,1,1))
row3<-ggarrange(plts$LOC115494624,plts$PFKP,plts$PHETA1,plts$SDC1,plts$SIX2,NULL, nrow=1,widths=c(1,1,1,1,1,1))
ggarrange(row1,row2,row3,nrow=3)
#fpkm %>% filter(node=='dn') %>% filter(gene_id %in% core_genes)

# 7 - Chromosomal enrichments

## First lets just look at the data and see if any chromosome module pairings stand out

In [ ]:
# Specify the lib path
lib_path<-paste(getwd(),'/hana_reanalysis_lib',sep='')
print(lib_path)

# Set the path
.libPaths(lib_path)
.libPaths()

In [ ]:
library(tidyverse)
library(ggpubr)

In [ ]:
#BiocManager::install("rtracklayer")
library(rtracklayer)

## Read in module memberships

In [ ]:
g_members <- readRDS('4_mod_membership_all_genes.rds') %>% filter(me_assigned != 'X') %>% 
    filter(me_assigned!='X')
head(g_members)

## Read in the genome annotation, really just need the gene_ids 

In [ ]:
annot <- rtracklayer::import('/v-data2/matt_davenport/hana_reanalyses/hana_cm_formatted/genome/GCF_008822105.2_bTaeGut2.pat.W.v2_genomic.purged2.gtf') %>%
            as_tibble %>% 
            filter(type=='gene') %>% dplyr::select(one_of(c('gene_id','start', 'seqnames'))) %>% arrange(start) %>%
            mutate(chr=seqnames) %>% mutate(seqnames=NULL)
head(annot)

In [ ]:
g_mem_forest <- left_join(g_members,annot,by='gene_id') %>% arrange(start) %>% mutate(chr=chr %>% as.character) 
head(g_mem_forest)

In [ ]:
g_mem_forest$chr[which(grepl('NW_', g_mem_forest$chr))] <- 'NW_xxxxxx.x'

g_mem_forest <- g_mem_forest %>% arrange(chr,start) %>% 
                mutate(chr=factor(chr,levels=unique(chr))) %>% group_by(chr)
head(g_mem_forest)
unique(g_mem_forest$chr)

## first I need to know how many genes of each module on each chromosome

In [ ]:
#make some variables for convenience
mod_cols <- c('A','B','C','D','E','F','G','H','I','J','K','L')
names(mod_cols) <- mod_cols

mod_per_chr <- mod_cols %>%
    lapply(function(col){
        return(g_mem_forest %>% 
        filter(me_assigned == col) %>% dplyr::select(one_of('chr','gene_id')) %>% 
        group_by(chr) %>% summarize(count = n()) %>% ungroup)
        
    })

 mod_per_chr <- names(mod_per_chr) %>% lapply(function(nm){
     tbl <- mod_per_chr[[nm]]
     names(tbl) <- c('chr',nm)
     return(tbl)
 })

In [ ]:
mod_chr_tbl<- purrr::reduce(mod_per_chr,left_join,by='chr')
mod_chr_tbl[is.na(mod_chr_tbl)] <- 0
mod_chr_tbl <- mod_chr_tbl %>% mutate(total = A+B+C+D+E+F+G+H+I+J+K+L) %>% arrange(-total)

## Now interms of random chance

In [ ]:
fold_expected <- mod_chr_tbl %>% mutate_at(c(mod_cols,'total'),
                                           function(col){
                                               return(col/sum(col)/(mod_chr_tbl$total/sum(mod_chr_tbl$total)))
                                           }) %>% dplyr::select(-one_of('total'))

In [ ]:
fold_expected <- fold_expected %>% pivot_longer(one_of(mod_cols),names_to='module',values_to='fold_expected')

In [ ]:
library(RColorBrewer)

options(repr.plot.width=25, repr.plot.height=10)


fold_expected %>% 
    ggplot(aes(x=chr,y=fold_expected,fill=module)) + 
        geom_hline(yintercept=1,color='black',size=3) + 
        geom_point(shape=21,size=10) + 
        scale_fill_brewer('Module',palette='Set3')+
        theme_grey(base_size=25) +
        scale_y_continuous('Fold Enrichment',breaks=c(1,5,10))+
        scale_x_discrete('\nChromosome')+
        theme(axis.text.x=element_text(angle=90))+ 
        guides(fill = guide_legend(override.aes = list(size=10)))

## Wonderful, but how to I know if a value is significant? I need to bootstrap some bonferoni corrected alpha cutoffs for each pairing

### Takes a long time

In [ ]:
boot_straps <- seq(1:25000) %>% lapply(function(sq){paste('boot_me_',sq,sep='')})
names(boot_straps) <- boot_straps 

boot_g_mem_forest <- g_mem_forest

bootstrapped_mes <- boot_straps %>% 
    lapply(function(boot){
        col <- sample(g_mem_forest$me_assigned,length(g_mem_forest$gene_id),replace=FALSE)
        return(col)
        
    }) %>% as_tibble %>% mutate(gene_id = g_mem_forest$gene_id)

bootstrapped_mes <- g_mem_forest %>% dplyr::select(one_of('chr','gene_id')) %>% left_join(bootstrapped_mes,by='gene_id')
head(bootstrapped_mes)

## Now I need to calculate the fold expected for each pairing within each bootstrap

### takes a long time

In [ ]:
chromos <- fold_expected$chr %>% unique
boots<-names(boot_straps)
mods <- mod_cols

a<-chromos %>% lapply(function(chromo){
    
    chromosome_tbl <- bootstrapped_mes %>% filter(chr==chromo) %>% 
        dplyr::select(starts_with('boot_me'),chr) %>% ungroup
    
    boot_straps %>% lapply(function(boot){
        
        boot_tbl <- chromosome_tbl %>% dplyr::select(all_of(boot),'chr')
            
        fold_expected <- mods %>% lapply(function(mod){
            frac_mod <- sum(bootstrapped_mes[[boot]]==mod)/length(bootstrapped_mes[[boot]])
            frac_chrom <-  sum(bootstrapped_mes$chr==chromo)/length(bootstrapped_mes$chr)
            frac_expected <- frac_mod * frac_chrom
            frac_got <- sum(boot_tbl[[boot]]==mod)/length(bootstrapped_mes$chr)
            
            fold_expected <- frac_got/frac_expected
            
        }) %>% as_tibble 
     }) %>% purrr::reduce(rbind) %>% 
        pivot_longer(everything(),names_to='mod',values_to='fold_expected') %>% mutate(chr=chromo %>% as.character)
}) %>% purrr::reduce(full_join,by = c("mod", "fold_expected", "chr"))

head(a)



## And now to estimate the alpha cutoff

In [ ]:
generate_alpha <- function(chrom,modu){
    a %>% filter(mod ==modu) %>%
        filter(chr==chrom) %>% 
        select(fold_expected) %>% 
        as.matrix %>% quantile(.99986979166)
}



generate_alpha('NC_045000.1','A')
generate_alpha('NC_045000.1','D')

In [ ]:
fold_expected$boot_alpha <- map2(fold_expected$chr,
                                 fold_expected$module,
                                 generate_alpha) #%>% as.numeric

fold_expected <- fold_expected %>% mutate(is_sig = boot_alpha <= fold_expected )
head(fold_expected)

## Now lets do some plotting

In [ ]:
actual_chrs <- c('1','1a','2','3','4','4a','5','6','7','8','9','10','11','12','13','14','15',
'17','18','19','20','21','22','23','24','25','26','27','28','Z','W','29','other')%>%as.matrix

In [ ]:
options(repr.plot.width=10, repr.plot.height=13
       )

g_legend<-function(a.gplot){
  tmp <- ggplot_gtable(ggplot_build(a.gplot))
  leg <- which(sapply(tmp$grobs, function(x) x$name) == "guide-box")
  legend <- tmp$grobs[[leg]]
  return(legend)} 
                      
enrich_plt<- fold_expected %>% 
    ggplot(aes(y=chr,x=fold_expected,
               fill=module,color=is_sig)) + 
        geom_vline(xintercept=1,color='black',size=3) + 
        geom_point(data=fold_expected %>% filter(is_sig==FALSE),shape=21,stroke=2,alpha=.3,size=7,show.legend = FALSE) + 
        geom_point(data=fold_expected %>% filter(is_sig==TRUE),shape=21,stroke=2,size=10) + 
        scale_fill_brewer('Module',palette='Set3')+
        scale_color_manual('pBonf\n<= 0.05', values=c('grey','black'))+
        theme_grey(base_size=25) +
        scale_x_continuous('Fold Enrichment',breaks=c(1,5,10,15),limits=c(0,15))+
        scale_y_discrete('\nChromosome',labels=rev(actual_chrs),limits=rev(levels(fold_expected$chr)))+
        guides(fill = guide_legend(override.aes = list(size=10)))

lgnd <- g_legend(enrich_plt)
enrich_plt <- enrich_plt +theme(legend.position='none')
                      enrich_plt

In [ ]:
names(actual_chrs)<-NULL
actual_chrs_m <-actual_chrs %>% as.matrix

## Ok now I want to make a plot for each chromosome w/ sig enrichment showing the position of the genes from enriched modules

## Save the NW for last

In [ ]:
enriched_mod_tbl <- fold_expected %>% filter(is_sig == TRUE) %>% 
    filter(chr!='NW_xxxxxx.x') %>% select(chr,module) #%>% t %>% as.data.frame

enriched_mod_tbl 

In [ ]:
gene_list <- map2(enriched_mod_tbl$chr, enriched_mod_tbl$module,
     function(chrom, modu){
         g_mem_forest %>% filter(chr==chrom) %>% filter(me_assigned == modu) %>% select(gene_id,chr,me_assigned,start)
     }) %>% purrr::reduce(rbind)# %>% as_tibble

In [ ]:
head(gene_list)

In [ ]:
g_mem_forest %>% ggplot(aes(y=start/1000000,x=chr,color=me_assigned))+
    geom_jitter(position = position_jitter(width = 0.25, height = 0),alpha=1)+
    scale_color_brewer('Module',palette='Set3')+
    theme_grey(base_size=25)+
    theme(axis.text.x=element_text(angle=90))+ 
    xlab('\nChromsome')+ylab('Start Position (MB)\n')+
    guides(color = guide_legend(override.aes = list(size=10)))

In [ ]:
chr_by_mod<- g_mem_forest %>% 
    ggplot(aes(y=chr,x=1,fill=me_assigned)) +
    geom_bar(position="fill", stat="identity")+ 
    scale_fill_brewer('Module',palette='Set3')+
    theme_minimal(base_size=25)+
    theme(axis.text.x=element_text(colour = 'white'),
          axis.text.y=element_blank(),
          axis.ticks=element_blank()) + 
    xlab('') + ylab('') +
    theme(legend.position='none')+scale_y_discrete('',labels=rev(actual_chrs),limits=rev(levels(fold_expected$chr)))
    

chr_by_mod

In [ ]:
library(ggpubr)
options(repr.plot.width=18, repr.plot.height=13
       )
ggarrange(enrich_plt,chr_by_mod,NULL,lgnd,nrow=1,widths=c(1,.25,.07,.2))

## Ok, lets look at E continuously and pull out the sex chromsomes

In [ ]:
E_tbl <- g_mem_forest %>% ungroup %>% select(E,gene_id, me_assigned, chr) %>%
    mutate(is_w=chr=='NC_045028.1') %>%
    mutate(is_z=chr== 'NC_045027.1')

E_tbl$sex_chromosome = map2(E_tbl$is_w,E_tbl$is_z,function(.x,.y){
    if (!.x & !.y) {
        return('Somatic')
    }else if (.x){
        return('W')
    }else if (.y){
        return('Z')
    }
}) %>% unlist %>% as.factor


In [ ]:
options(repr.plot.width=11, repr.plot.height=4)

E_tbl %>% 
    ggplot(aes(x=E, fill=sex_chromosome, group= sex_chromosome)) + 
    geom_density(alpha=.7) + theme_grey(base_size=25) +
    scale_fill_manual('Chromosome',values = c('dark grey','magenta','cyan')) + 
    ylab('Density') +
    theme(axis.text.y=element_blank(),
          axis.ticks.y=element_blank())

## I'd love to take a look a the interaction of MEG E and MEG G in the gene space.

In [ ]:
g_mem_forest %>%ungroup() %>% select(E,G,gene_id,me_assigned) %>% 
    ggplot(aes(x=G^2,y=E^2, fill=me_assigned)) +
    geom_point(shape=21,stroke=.1,size=3) +
    geom_label(data=g_mem_forest %>%ungroup() %>% select(E,G,gene_id,me_assigned) %>% filter(.375<G^2) %>% filter(.25<E^2),
                   aes(label=gene_id),size=8)+
    theme_grey(base_size=25)+
    scale_fill_brewer('Module',palette = 'Set3')